![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [121]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

student_id      city  ...  training_hours job_change
0        8949  city_103  ...              36        1.0
1       29725   city_40  ...              47        0.0
2       11561   city_21  ...              83        0.0
3       33241  city_115  ...              52        1.0
4         666  city_162  ...               8        0.0

[5 rows x 14 columns]

In [122]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [123]:
# the number of null values per column
ds_jobs_transformed.isna().sum().sort_values(ascending = False)

company_type              6140
company_size              5938
gender                    4508
major_discipline          2813
education_level            460
last_new_job               423
enrolled_university        386
experience                  65
student_id                   0
city                         0
city_development_index       0
relevant_experience          0
training_hours               0
job_change                   0
dtype: int64

In [124]:
# Define mappings for two-factor categories
two_factor_cats = {
    'relevant_experience': {'Has relevant experience': True, 'No relevant experience': False},
    'job_change': {1.0: True, 0.0: False}
}

In [125]:
# Define mappings for ordinal categories
ordered_cats = {
    'relevant_experience': ['No relevant experience', 'Has relevant experience'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}



To check and compare memory usage of the two dataframes


In [126]:
# Transform the dataset
for col in ds_jobs_transformed.columns:
    if col in two_factor_cats:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].map(two_factor_cats[col])
    elif col in ordered_cats:
        category = pd.CategoricalDtype(categories=ordered_cats[col], ordered=True)
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype(category)
    elif ds_jobs_transformed[col].dtype == 'int64':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    elif ds_jobs_transformed[col].dtype == 'float64':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    else:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

In [127]:
# Dataframe 1 memory usage before filtering
ds_jobs_memory_usage = ds_jobs.memory_usage(deep=True).sum()
print(f'Total memory usage of the ds_jobs dataframe is {ds_jobs_memory_usage} bytes')

# Dataframe 2 memory usage before filtering
ds_jobs_clean_memory_usage = ds_jobs_transformed.memory_usage(deep=True).sum()
print(f'Total memory usage of the ds_jobs_transformed dataframe after optimization is {ds_jobs_clean_memory_usage} bytes')

# Calculate percentage decrease in memory usage
diff1 = ds_jobs_memory_usage - ds_jobs_clean_memory_usage
percent_decrease1 = (diff1 / ds_jobs_memory_usage) * 100
print(f'After optimization, I achieved a {percent_decrease1:.2f}% decrease in memory usage.')


Total memory usage of the ds_jobs dataframe is 12391705 bytes
Total memory usage of the ds_jobs_transformed dataframe after optimization is 419965 bytes
After optimization, I achieved a 96.61% decrease in memory usage.


In [128]:
# Filter the DataFrame
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'].cat.codes >= ordered_cats['experience'].index('10')) & 
    (ds_jobs_transformed['company_size'].cat.codes >= ordered_cats['company_size'].index('1000-4999'))
]

# Dataframe 2 memory usage after filtering
ds_jobs_clean_memory_usage_filtered = ds_jobs_transformed.memory_usage(deep=True).sum()
print(f'Total memory usage of the ds_jobs_transformed dataframe after filtering is {ds_jobs_clean_memory_usage_filtered} bytes')

# Calculate percentage decrease in memory usage after filtering
diff2 = ds_jobs_memory_usage - ds_jobs_clean_memory_usage_filtered
percent_decrease2 = (diff2 / ds_jobs_memory_usage) * 100
print(f'Upon filtering, I achieved a {percent_decrease2:.2f}% decrease in memory usage.')

Total memory usage of the ds_jobs_transformed dataframe after filtering is 81348 bytes
Upon filtering, I achieved a 99.34% decrease in memory usage.
